# Interval arithmetic

We will work with [PyInterval](https://pyinterval.readthedocs.io/en/latest/index.html). To install PyInterval in Anaconda, use:

```bash
conda install -c conda-forge pyinterval
```

## Some motivating examples

### A recursion formula

Consider the recursion formula
$$x_{n+1} = x_n^2$$ ($n = 0, 1, 2, \ldots$), and suppose that $x_0 = 1 − 10^{−21}$. We seek $x_{75}$.

Can we compute $x_{75}$ with standard floating point arithmetic?

In [1]:
x = [1-10**(-21)]
for _ in range(75):
    x += [x[-1]**2]
    
print(f'x[0]={x[0]}, x[1]={x[1]}, ..., x[75]={x[75]}')

x[0]=1.0, x[1]=1.0, ..., x[75]=1.0


Performing the computation with 10-place arithmetic, we obtain the approximate values.

In [2]:
import decimal as dec
dec.getcontext().prec = 10

In [3]:
x = [1-dec.Decimal(10)**(-21)]
for _ in range(75):
    x += [x[-1]**2]
    

In [4]:
print(f'x[0]={x[0]}, x[1]={x[1]}, ..., x[75]={x[75]}')

x[0]=0.999999999999999999999, x[1]=0.9999999999999999999980000000, ..., x[75]=3.915780683799750301999125205E-17


In [5]:
dec.getcontext().prec = 20

x = [1-dec.Decimal(10)**(-21)]
for _ in range(75):
    x += [x[-1]**2]
    
print(f'x[0]={x[0]}, x[1]={x[1]}, ..., x[75]={x[75]}')

x[0]=1.0000000000000000000, x[1]=1.0000000000000000000, ..., x[75]=1.0000000000000000000


In [6]:
dec.getcontext().prec = 30

x = [1-dec.Decimal(10)**(-21)]
for _ in range(75):
    x += [x[-1]**2]
    
print(f'x[0]={x[0]}, x[1]={x[1]}, ..., x[75]={x[75]}')

x[0]=0.999999999999999999999, x[1]=0.999999999999999999998000000000, ..., x[75]=3.91578068379999471438826301494E-17


### Function evaluation

Evaluate the following function:
$$f = 333.75b^6 + a^2(11a^2b^2-b^6-121b^4-2)+5.5b^8+a/(2b)$$
with $a=77617.0$ and $b=33096.0$.

In [7]:
f = lambda a,b: dec.Decimal('333.75')*(b**6) +\
                (a**2)*(11*(a**2)*(b**2)-(b**6)-121*(b**4)-2)+\
                dec.Decimal('5.5')*(b**8)+a/(2*b)

In [8]:
a = dec.Decimal(77617)
b = dec.Decimal(33096)

fs = {}
for p in range(1,70,5):
    dec.getcontext().prec = p
    fs[p] = f(a,b)
    print(f"precision = {p}: f = {fs[p]}")
    

precision = 1: f = 1
precision = 6: f = -1.00000E+31
precision = 11: f = -3.0000000000E+26
precision = 16: f = 1.000000000000000E+21
precision = 21: f = 10000000000000001.1726
precision = 26: f = 1.1726039400531786318588349
precision = 31: f = -999998.8273960599468213681411651
precision = 36: f = 21.1726039400531786318588349045201837
precision = 41: f = -0.8273960599468213681411650954798162919990
precision = 46: f = -0.827396059946821368141165095479816291999033116
precision = 51: f = -0.82739605994682136814116509547981629199903311578438
precision = 56: f = -0.8273960599468213681411650954798162919990331157843848199
precision = 61: f = -0.827396059946821368141165095479816291999033115784384819917815
precision = 66: f = -0.82739605994682136814116509547981629199903311578438481991781484167


What happens with floating point arithmetic?

In [9]:
f = lambda a,b: 333.75*(b**6) +\
                (a**2)*(11*(a**2)*(b**2)-(b**6)-121*(b**4)-2)+\
                5.5*(b**8)+a/(2*b)

In [10]:
a = 77617.0
b = 33096.0

In [11]:
f(a,b)

-1.1805916207174113e+21

There are cases that no finite precision gives acceptable results:

In [12]:
from math import sin, acos

sin(2*acos(0))==0

False

- In many cases it is true that by carrying enough places a result of arbitrarily high accuracy can be found in any computation involving only a finite number of real arithmetic operations beginning with exactly known real numbers. 
- However, it is often prohibitively difficult to tell in advance of a computation how many places must be carried to guarantee results of required accuracy

> These and other problems arise because any finite precision number is an imprecise representation of a real number.

## The interval Number System

We focus on closed intervals only. A (closed) interval is a set:
$$[a,b] = \{x \in \mathbb{R}: a \leq x \leq b\}$$

### Intervals in PyInterval

In [13]:
from interval import interval

Intervals are immutable objects that can be created by specifying their connected components:

In [14]:
k = interval([0, 1], [2, 3], [10, 15])
k

interval([0.0, 1.0], [2.0, 3.0], [10.0, 15.0])

In [15]:
interval[1, 2]

interval([1.0, 2.0])

represents the mathematical interval [1, 2], not be confused with the union of the one-point intervals {1} and {2}:

In [16]:
interval(1,2)

interval([1.0], [2.0])

An interval consisting of only one number can be instantiated with either forms:

In [17]:
interval(1), interval[1]

(interval([1.0]), interval([1.0]))

An empty interval has no components:

In [18]:
interval()

interval()

In [19]:
len(interval())

0

In [20]:
len(interval[1, 2])

1

In [21]:
len(interval(1, 2))

2

In [22]:
[x for x in interval([1, 2], 3)]

[(1.0, 2.0), (3.0, 3.0)]

In [23]:
[x for x in interval([1, 2], 3).components]

[interval([1.0, 2.0]), interval([3.0])]

### Endpoints
Endpoints are denoted as:
$$X=[\underline{X}, \overline{X}]$$

Two intervals are equal if they have the same endpoints:
$$X=Y \Leftrightarrow \underline{X}=\underline{Y} \wedge \overline{X}=\overline{Y}$$

In [24]:
x = interval([1, 2], 3)

In [25]:
x[0].inf

1.0

In [26]:
x[1].sup

3.0

In [27]:
interval([1, 2], 3).extrema

interval([1.0], [2.0], [3.0])

In [28]:
interval[1,2] == interval([1,2])

True

#### Degenerate intervals
$$x = [x,x]$$

In [29]:
#warning!
0 == interval[0,0]

False

### Set operations on intervals

In [30]:
# intersection
interval[1, 4] & interval[2, 5]

interval([2.0, 4.0])

In [31]:
# union
interval[1, 4] | interval[2, 5]

interval([1.0, 5.0])

In general, the union of the two intervals is not an interval

In [32]:
interval[1, 2] | interval[4, 5]

interval([1.0, 2.0], [4.0, 5.0])

However, a *interval hull* is always an interval

In [33]:
interval.hull((interval[1,2], interval[4,5]))

interval([1.0, 5.0])

> Information is lost when we replace classical union with interval hull, but the latter is easier to work with, and the lost information is sometimes not critical.

In [34]:
0 in interval[-1, 1]

True

In [35]:
0 in interval[1, 2]

False

In [36]:
interval[1, 2] in interval[0, 3]

True

In [37]:
interval[1, 2] in interval[1.5, 3]

False

#### Importance of intersection

If we have two intervals containing a result of interest, then the intersection, which may be narrower, also contains the result.

**Example.** 
Suppose two people make independent measurements of the same physical quantity $q$. One finds that $q = 10.3$ with a measurement error less than 0.2. The other finds that $q = 10.4$ with an error less than 0.2. We can represent these measurements as the intervals $X = [10.1, 10.5]$ and $Y = [10.2, 10.6]$, respectively. Since $q$ lies in both, it also lies in $X \cap Y = [10.2, 10.5]$. An empty intersection would imply that at least one of the measurements is wrong

### Measures of intervals

#### Width

In [38]:
def width(ic):
    return ic.sup - ic.inf

In [39]:
x = interval([-2, 3])
width(x[0])

5.0

#### Absolute value

In [40]:
def abs_val(ic):
    return max(abs(ic.inf), abs(ic.sup))

In [41]:
abs_val(x[0])

3.0

#### Midpoint

In [42]:
interval([1, 2], 3).midpoint

interval([1.5], [3.0])

### Extending component functions

In [43]:
@interval.function
def width(ic):
    return interval(ic.sup - ic.inf)

In [44]:
width(x)

interval([5.0])

In [45]:
@interval.function
def abs_val(ic):
    return interval(max(abs(ic.inf), abs(ic.sup)))

In [46]:
abs_val(x)

interval([3.0])

In [47]:
@interval.function
def reverse(c):
    return interval[-c.sup, -c.inf]

reverse(interval([1, 2], 3))

interval([-3.0], [-2.0, -1.0])

In [48]:
@interval.function
def mirror(c):
    return interval[-c.sup, -c.inf] | interval(c)

In [49]:
mirror(interval([1, 2], 3))

interval([-3.0], [-2.0, -1.0], [1.0, 2.0], [3.0])

### Order relations

- different possible definitions.

Standard definition: $X < Y$ means $\overline{X} < \underline{Y}$

In PyInterval: $X < Y$ means $\underline{X} < \underline{Y}$

In [50]:
x = interval[1,2]
y = interval[3,4]

x < y

True

In [51]:
x = interval[3.5,5]
y = interval[3,4]

x < y

False

In [52]:
x = interval[3,5]
y = interval[3,4]

x < y

False

## Operations with intervals

### Sum

$X+Y = \{x+y: x\in X, y \in Y\} = [\underline{X}+\underline{Y}, \overline{X}+\overline{Y}]$

In [53]:
interval[0, 2] + interval[-1, 1]

interval([-1.0, 3.0])

### Scalar multiplication

$c\cdot X = \{c\cdot x: x \in X\} =$
- $[c\underline{X}, c\overline{X}]$ if $c\geq 0$
- $[c\overline{X}, c\underline{X}]$ if $c< 0$

In [54]:
2*interval([-2, 3])

interval([-4.0, 6.0])

In [55]:
-2*interval([-2,3])

interval([-6.0, 4.0])

### Subtraction

$X-Y = X+ (-1)\cdot Y = [\underline{X}-\overline{Y}, \overline{X}-\underline{Y}]$

In [56]:
interval[5, 7] - interval[1, 2]

interval([3.0, 6.0])

In [57]:
interval[5, 7] + (-1) * interval[1, 2]

interval([3.0, 6.0])

In [58]:
interval[5, 7] - interval[5,7]

interval([-2.0, 2.0])

In general, $X-X \neq 0$. Why?

### Multiplication

$XY = [\min S, \max S]$ where $S=\{\underline{X}\underline{Y}, \underline{X}\overline{Y}, \overline{X}\underline{Y} ,\overline{X}\overline{Y}\}$

In [59]:
interval[3,2] * interval[-2,2]

interval([-6.0, 6.0])

In [60]:
interval[2,3] * interval[0]

interval([0.0])

In [61]:
from interval import inf

In [62]:
interval[0, 2] * interval[4, inf]

interval([-inf, inf])

### Reciprocal

$1/X = \left[1/\overline{X}, 1/\underline{X}\right]$ provided that $0 \notin X$

In [63]:
1/interval[2,3]

interval([0.3333333333333333, 0.5])

In [64]:
1/interval[-1,1]

interval([-inf, -1.0], [1.0, inf])

### Division

$X/Y = X * (1/Y)$ provided that $0 \notin Y$

In [65]:
interval[1, 4] / interval[-3, -2]

interval([-2.0, -0.3333333333333333])

In [66]:
interval[1, 4] / interval[-3, 3]

interval([-inf, -0.3333333333333333], [0.3333333333333333, inf])

## Examples

### Area of an imprecise rectangle

- Suppose we wish to calculate the area a of a rectangle from direct measurements of its side lengths $l$ and $w$. 
- Use of a standard meter stick shows that $l$ equals 1 m to within an uncertainty of 0.0005 m (i.e., to within half the least count measurement of 1 mm).

Since
$$0.9995 ≤ l ≤ 1.0005$$
we construct an interval $L = [0.9995, 1.0005]$ to represent this side length.

- Measuring again, we find that $w = 2$ (nominally). Hence, the remaining side should be represented by the interval $W = [1.9995, 2.0005]$.

$$
\begin{aligned}
  A = L · W \\
    = [0.9995, 1.0005] · [1.9995, 2.0005]\\
    = [0.9995 · 1.9995 , 1.0005 · 2.0005]\\
    = [1.99850025 , 2.00150025] \textrm{m}^2
\end{aligned}
$$

- Suppose it is not necessary to know the final answer to eight places. 
- We are therefore tempted to round off the endpoints of $A$. 
- This is indeed possible, but we must be careful because the true value a of the product $lw$ can fall anywhere within the interval that is specified exactly by $LW$. 

- Suppose the true value of $a$ is $a = 1.99850026$ m².
-  if we were to round the endpoints of $A$ both *upward* by one digit to obtain $A' = [1.9985003 , 2.0015003]$ m²
- this new interval $A'$ would not contain the actual value!

- we will apply *outward rounding*: we will round in such a way that the left endpoint moves to the left (on the number line) and the right endpoint moves to the right. 
- The resulting interval contains the one we started with and hence still has a as a member. 
- In the present example, we could round outwardly to the nearest square millimeter and obtain the interval $A'' = [1.998, 2.002]$ m².

##### Exercise
The dimensions of a rectangular box are measured as $w = 7.2 ± 0.1$, $l = 14.9 ± 0.1$, and $h = 405.6 ± 0.2$. Find several intervals containing the volume of the box.

### Dependency effect

In [78]:
x = interval[-1, 2]
print("x·x-2x   = ", x*x - 2*x)
print("x²-2x    = ", x**2 - 2*x)
print("x·(x-2)  = ", x*(x-2))
print("(x-1)²-1 = ", (x-1)**2-1)

x·x-2x   =  interval([-6.0, 6.0])
x²-2x    =  interval([-4.0, 6.0])
x·(x-2)  =  interval([-6.0, 3.0])
(x-1)²-1 =  interval([-1.0, 3.0])


## Properties of interval arithmetic

### Commutativity 

- $X+Y=Y+X$
- $XY = YX$

### Associativity

- $X + (Y+Z) = (X+Y)+Z$
- $X(YZ)=(XY)Z$

### Identity elements

- $0+X=X+0=X$
- $1·X=X·1=X$
- $0·X=X·0=0$

### Nonexistence of inverse elements

- $X-X = [\underline{X}-\overline{X}, \overline{X}-\underline{X}]$
- $X/X = [\underline{X}/\overline{X}, \overline{X}/\underline{X}]$ if $\underline{X}>0$, $[\overline{X}/\underline{X},\underline{X}/\overline{X}]$ if $\overline{X}<0$, undefined otherwise.

### Subdistributivity

- $X(Y+Z) \subseteq XY+XZ$

### Cancellation law

- $X+Z=Y+Z \Rightarrow X=Y$

but

- $ZX=ZY \nRightarrow X=Y$

## Interval functions

### Extension principle

Let $U$ and $V$ be two sets and $f : U \mapsto V$ a function.
Let $A \subseteq U$, then the *image* of $A$ under $f$ is given by 
$$ f(A) = \{f(x) | x \in A\}$$

It is possible to extend the concept of image to binary relations.

Let $U$ and $V$ be two sets and $R \subseteq U \times V$ a relation. Let $A \subseteq U$, then the composition of $A$ and $R$ is defined as:
$$ A \circ R = \{y \in V | \exists x : x\in A \wedge xRy\}$$

Since any function $f$ is a particular relation between $U$ and $V$, then the image of a function derived from the composition operator:
$$A \circ f = \{y \in V | \exists x: x\in A \wedge xfy \}$$
$$= \{y \in Y | \exists x: x\in A \wedge y=f(x) \}$$
$$= \{f(x) | x\in A \} = f(A)$$

The *united extension* of a function $f$ to intervals $X$ is
$$f(X)=\left[\min\{f(x):x\in X\}, \max\{f(x):x\in X\}\right]$$
when the definition makes sense.

### Square

$$X^2 = \{x^2:x \in X\}$$, i.e.
$$
X^{2}=\begin{cases}
\left[\underline{X}^{2},\overline{X}^{2}\right] & 0\leq\underline{X},\\
\left[\overline{X}^{2},\underline{X}^{2}\right] & \overline{X}\leq0,\\
\left[0,\max\left\{ \overline{X}^{2},\underline{X}^{2}\right\} \right] & 0\in X
\end{cases}
$$

In general, $X^2 \subseteq X·X$

> *Interval dependency* is a crucial consideration when using interval computations. It is a major reason why simply replacing floating point computations by intervals in an existing algorithm is not likely to lead to satisfactory results

### Monotonic functions

If $f$ is an increasing function, then
$$f(X)=[f(\underline{X}), f(\overline{X})]$$

![example](monotonic.png)

In [67]:
from interval import imath

In [26]:
imath.exp(interval[0, 1])

interval([1.0, 2.7182818284590455])

Similar arguments apply for decreasing functions.

### Interval extension of functions

A function $F$ is an *interval extension* of $f$ if, for all $x$:
$$F([x,x])=f(x)$$

- Interval extensions are not unique. Two different functions $F'$ and $F''$ can be interval extensions of the same function $f$.
- For example, $F(X)=X^2-2X$ and $G(X)=X·(X-2)$ are interval extensions of $f(x)=x^2-2x$ but, in general $F(X)\neq G(X)\neq f(X)$

- The united extension coincides with an interval extension where each argument occurs only once in the definition

$$H(X)=\frac{1}{4}-\left(X-\frac{1}{2}\right)^2=f(X)$$

#### Fundamental Theorem of Interval Analysis

$$f(X) \subseteq F(X)$$

## Intervals in pandas

- Intervals in pandas are quite limited in functionality
  - However, there are some extended features, such as open and semi-open intervals
- Integration with pyinterval is missing

In [14]:
import pandas as pd

In [16]:
iv = pd.Interval(left=0, right=5)
iv

Interval(0, 5, closed='right')

In [17]:
2.5 in iv

True

In [18]:
0 in iv

False

In [19]:
5 in iv

True

In [20]:
0.0001 in iv

True

In [21]:
iv.length

5

In [23]:
shifted_iv = iv + 3
shifted_iv

Interval(3, 8, closed='right')

In [26]:
extended_iv = iv * 10.0
extended_iv

Interval(0.0, 50.0, closed='right')

In [27]:
iv * extended_iv

TypeError: unsupported operand type(s) for *: 'pandas._libs.interval.Interval' and 'pandas._libs.interval.Interval'

In [28]:
year_2017 = pd.Interval(pd.Timestamp('2017-01-01 00:00:00'), \
                        pd.Timestamp('2018-01-01 00:00:00'), \
                        closed='left')

In [29]:
pd.Timestamp('2017-01-01 00:00') in year_2017

True

In [30]:
year_2017.length

Timedelta('365 days 00:00:00')

In [31]:
volume_1 = pd.Interval('Ant', 'Dog', closed='both')

ValueError: Only numeric, Timestamp and Timedelta endpoints are allowed when constructing an Interval.

### Interval methods

In [32]:
pd.Interval(0, 1, closed='right').is_empty

False

In [33]:
pd.Interval(0, 0, closed='right').is_empty

True

In [34]:
pd.Interval(0, 0, closed='both').is_empty

False

In [35]:
ivs = [pd.Interval(0, 0, closed='neither'), pd.Interval(1, 2, closed='neither')]
pd.arrays.IntervalArray(ivs).is_empty

array([ True, False])

In [37]:
ivs = [pd.Interval(0, 0, closed='neither'), np.nan]
pd.IntervalIndex(ivs).is_empty

array([ True, False])

In [40]:
i1 = pd.Interval(0, 2)
i2 = pd.Interval(1, 3)
i1.overlaps(i2)


True

In [39]:
i3 = pd.Interval(4, 5)
i1.overlaps(i3)

False

In [41]:
i4 = pd.Interval(0, 1, closed='both')
i5 = pd.Interval(1, 2, closed='both')
i4.overlaps(i5)

True

In [42]:
i6 = pd.Interval(1, 2, closed='neither')
i4.overlaps(i6)

False

### IntervalArray
Pandas array for interval data that are closed on the same side.

In [43]:
pd.arrays.IntervalArray([pd.Interval(0, 1), pd.Interval(1, 5)])

<IntervalArray>
[(0, 1], (1, 5]]
Length: 2, closed: right, dtype: interval[int64]

### IntervalIndex

In [68]:
df = pd.DataFrame({'A': [1, 2, 3, 4]}, 
                  index=pd.IntervalIndex.from_breaks([0, 1, 2, 3, 4]))

NameError: name 'pd' is not defined

In [45]:
df

,A
"(0, 1]",1
"(1, 2]",2
"(2, 3]",3
"(3, 4]",4


In [61]:
df.to_csv('intervals.csv')

In [71]:
pd.read_csv('intervals.csv', index_col=0)

,A
"(0, 1]",1
"(1, 2]",2
"(2, 3]",3
"(3, 4]",4


In [46]:
df.loc[2]

A    2
Name: (1, 2], dtype: int64

In [47]:
 df.loc[[2, 3]]

,A
"(1, 2]",2
"(2, 3]",3


In [48]:
df.loc[2.5]

A    3
Name: (2, 3], dtype: int64

In [49]:
 df.loc[[2.5, 3.5]]

,A
"(2, 3]",3
"(3, 4]",4


Selecting using an Interval will only return exact matches 

In [50]:
df.loc[pd.Interval(1, 2)]

A    2
Name: (1, 2], dtype: int64

In [51]:
df.loc[pd.Interval(0.5, 2.5)]

KeyError: Interval(0.5, 2.5, closed='right')

Selecting all Intervals that overlap a given Interval can be performed using the overlaps() method to create a boolean indexer.

In [53]:
idxr = df.index.overlaps(pd.Interval(0.5, 2.5))
idxr

array([ True,  True,  True, False])

In [54]:
df[idxr]

,A
"(0, 1]",1
"(1, 2]",2
"(2, 3]",3


### Binning data

In [57]:
c = pd.cut(range(4), bins=2)
c

[(-0.003, 1.5], (-0.003, 1.5], (1.5, 3.0], (1.5, 3.0]]
Categories (2, interval[float64]): [(-0.003, 1.5] < (1.5, 3.0]]

In [58]:
c.categories

IntervalIndex([(-0.003, 1.5], (1.5, 3.0]],
              closed='right',
              dtype='interval[float64]')

- cut() also accepts an IntervalIndex for its bins argument, which enables a useful pandas idiom.
- First, We call cut() with some data and bins set to a fixed number, to generate the bins. 
- Then, we pass the values of .categories as the bins argument in subsequent calls to cut(), supplying new data which will be binned into the same bins.

In [59]:
pd.cut([0, 3, 5, 1], bins=c.categories)

[(-0.003, 1.5], (1.5, 3.0], NaN, (-0.003, 1.5]]
Categories (2, interval[float64]): [(-0.003, 1.5] < (1.5, 3.0]]

### Generating ranges of intervals

In [60]:
pd.interval_range(start=0, end=5)

IntervalIndex([(0, 1], (1, 2], (2, 3], (3, 4], (4, 5]],
              closed='right',
              dtype='interval[int64]')